## Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader, TensorDataset, random_split

Microsoft Visual C++ Redistributable is not installed, this may lead to the DLL load failure.
                 It can be downloaded at https://aka.ms/vs/16/release/vc_redist.x64.exe


## Data Preparation

In [2]:
df=pd.read_csv("housing.csv")
df.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

In [3]:
df.replace([np.inf, -np.inf], np.nan)
df.dropna(inplace=True) 

In [4]:
targets = df[["median_house_value"]].to_numpy()
inputs = df.drop(["median_house_value", 'ocean_proximity'], axis=1).to_numpy()  

In [5]:
inputs = torch.from_numpy(inputs)
inputs = inputs.float()
targets = torch.from_numpy(targets)
targets = targets.float()

In [6]:
inputs_mean = torch.mean(inputs, axis=0)
targets_mean = torch.mean(targets, axis=0)
inputs = inputs / inputs_mean  
targets = targets / targets_mean 

In [7]:
dataset = TensorDataset(inputs, targets)

In [8]:
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)

## Define Two Models

In [9]:
model1 = nn.Sequential(nn.Linear(8, 4),
                      nn.ReLU(),

                      nn.Linear(4, 2),
                      nn.Softmax(),

                      
                      nn.Linear(2, 1),
                      nn.Sigmoid())

model2 = nn.Sequential(nn.Linear(8, 4),
                      nn.Softmax(),

                      nn.Linear(4, 1),
                      nn.ReLU())

In [11]:
# Utility function to train the model
def fit(num_epochs, model, criterion, optimizer, train_loader):
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        # Train with batches of data
        for xb,yb in train_loader:
            # 1. Generate predictions
            pred = model(xb)
            # 2. Calculate loss
            loss = criterion(pred, yb)
            # 3. Compute gradients
            loss.backward()
            # 4. Update parameters using gradients
            optimizer.step()
            # 5. Reset the gradients to zero
            optimizer.zero_grad()
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

## Fit Model1

In [12]:
# Define loss function
criterion = F.mse_loss
# Define optimizer
optimizer = torch.optim.SGD(model1.parameters(), lr=1e-4)
fit(100, model1, criterion, optimizer, train_loader)

C:\Users\user\Anaconda3\lib\site-packages\torch\nn\modules\container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [10/100], Loss: 0.5707
Epoch [20/100], Loss: 0.3706
Epoch [30/100], Loss: 0.8261
Epoch [40/100], Loss: 1.1195
Epoch [50/100], Loss: 0.4619
Epoch [60/100], Loss: 0.3027
Epoch [70/100], Loss: 0.4032
Epoch [80/100], Loss: 0.2036
Epoch [90/100], Loss: 0.6899
Epoch [100/100], Loss: 0.3238


## Fit Model2

In [13]:
# Define loss function
criterion = F.mse_loss
# Define optimizer
optimizer = torch.optim.SGD(model2.parameters(), lr=1e-4)
fit(100, model2, criterion, optimizer, train_loader)

Epoch [10/100], Loss: 0.3879
Epoch [20/100], Loss: 0.4310
Epoch [30/100], Loss: 0.3969
Epoch [40/100], Loss: 0.2582
Epoch [50/100], Loss: 0.2270
Epoch [60/100], Loss: 0.1945
Epoch [70/100], Loss: 0.2667
Epoch [80/100], Loss: 0.2581
Epoch [90/100], Loss: 0.5395
Epoch [100/100], Loss: 0.4476
